In [1]:
!pip install folium pandas openpyxl geopy

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from geopy.geocoders import Nominatim
import time

In [11]:
def load_and_process_pincode_data(file_path):
    try:
        # Read the Excel file
        df = pd.read_excel(file_path)
        
        # Ensure column names are correctly recognized
        # Convert column names to string and strip whitespace
        df.columns = [str(col).strip() for col in df.columns]
        
        # Check if the expected pincode column exists
        if 'pincode' not in df.columns:
            # Try to find alternative column names
            pincode_alternatives = ['pincode', 'PINCODE', 'Pincode', 'PIN', 'pin', 'Pin Code', 'PIN CODE', 'Pin code of Bangalore Residence ']
            
            # Find matching column
            pincode_col = None
            for alt in pincode_alternatives:
                if alt in df.columns:
                    pincode_col = alt
                    break
            
            # Rename column if found
            if pincode_col:
                df.rename(columns={pincode_col: 'pincode'}, inplace=True)
            else:
                # If no pincode column found, use the first column
                df.rename(columns={df.columns[0]: 'pincode'}, inplace=True)
        
        # Ensure pincode is treated as string
        df['pincode'] = df['pincode'].astype(str)
        
        # Count occurrences of each pincode
        pincode_counts = df['pincode'].value_counts().reset_index()
        pincode_counts.columns = ['pincode', 'count']
        
        return pincode_counts
    
    except Exception as e:
        print(f"Error loading file: {e}")
        return None

# Load your Excel file
df = load_and_process_pincode_data('bubbleplot_EMP.xlsx')

# Display the processed data
if df is not None:
    print(df.head())
else:
    print("Failed to load data. Please check your file.")


  pincode  count
0  560068     23
1  560076     17
2  560025     16
3  560066     15
4  560078     13


In [12]:
def geocode_pincodes(df, country='India'):
    """
    Convert pincodes to latitude and longitude using Nominatim
    Args:
        df: DataFrame with a 'pincode' column
        country: Country to use for geocoding (default: India)
    Returns:
        DataFrame with 'latitude' and 'longitude' columns added
    """
    # Initialize geocoder
    geolocator = Nominatim(user_agent="pincode_mapper")
    
    # Create empty columns for latitude and longitude
    df['latitude'] = None
    df['longitude'] = None
    
    # Geocode each pincode
    for idx, row in df.iterrows():
        pincode = row['pincode']
        try:
            # Query with pincode and country
            query = f"{pincode}, {country}"
            location = geolocator.geocode(query)
            
            if location:
                df.at[idx, 'latitude'] = location.latitude
                df.at[idx, 'longitude'] = location.longitude
                print(f"Geocoded {pincode}: {location.latitude}, {location.longitude}")
            else:
                print(f"Could not geocode {pincode}")
            
            # Sleep to avoid hitting API limits
            time.sleep(1)
        except Exception as e:
            print(f"Error geocoding {pincode}: {e}")
            time.sleep(1)
            continue
    
    # Drop rows with missing coordinates
    df_clean = df.dropna(subset=['latitude', 'longitude'])
    print(f"Successfully geocoded {len(df_clean)} out of {len(df)} pincodes")
    
    return df_clean

# Geocode the pincodes
# Note: This may take time depending on the number of pincodes
geocoded_df = geocode_pincodes(df)

# Show the geocoded data
geocoded_df.head()


Geocoded 560068: 12.8986856, 77.63657871843411
Geocoded 560076: 12.8826231, 77.6040474819001
Geocoded 560025: 12.963021000000001, 77.60357145087298
Geocoded 560066: 12.96684925, 77.75164275408477
Geocoded 560078: 12.89688445, 77.58255091382298
Geocoded 560027: 12.961163086021505, 77.58623299892473
Geocoded 560037: 12.9623272, 77.6972859093634
Geocoded 560035: 12.89793005, 77.70092991846323
Geocoded 560100: 12.8502428, 77.6545244022015
Geocoded 560067: 12.9455662, 77.82131801897232
Geocoded 560102: 12.91437835, 77.64325647698988
Geocoded 560087: 12.92446915, 77.72919469986229
Geocoded 560048: 12.98904855, 77.70673151352824
Geocoded 560034: 12.9255706, 77.6343792992431
Geocoded 560085: 12.93047605, 77.54532588798457
Geocoded 560016: 13.01969, 77.67737742671628
Geocoded 560017: 12.954811150000001, 77.66063780143375
Geocoded 560061: 12.90416415, 77.53994904955654
Geocoded 560036: 13.016153150000001, 77.69816482402385
Geocoded 560077: 13.060806249999999, 77.65483324435313
Geocoded 560103: 1

,pincode,count,latitude,longitude
0,560068,23,12.898686,77.636579
1,560076,17,12.882623,77.604047
2,560025,16,12.963021,77.603571
3,560066,15,12.966849,77.751643
4,560078,13,12.896884,77.582551


In [13]:
def create_folium_map(df, tiles='OpenStreetMap'):
    """
    Create a Folium bubble map with the geocoded pincode data
    Args:
        df: DataFrame with 'latitude', 'longitude', 'pincode', and 'count' columns
        tiles: Map tile style (default: 'OpenStreetMap')
    Returns:
        Folium map object
    """
    # Calculate the center of the map
    center_lat = df['latitude'].mean()
    center_lon = df['longitude'].mean()
    
    # Create a map with specified tile style
    m = folium.Map(location=[center_lat, center_lon], 
                  zoom_start=11, 
                  tiles=tiles)
    
    # Add a marker cluster for better performance with many points
    marker_cluster = MarkerCluster().add_to(m)
    
    # Calculate the max count for scaling
    max_count = df['count'].max()
    
    # Add markers for each pincode
    for idx, row in df.iterrows():
        # Calculate bubble size based on count (scaled)
        bubble_radius = min(50, max(5, row['count'] / max_count * 30))
        
        # Determine color intensity based on count
        # Higher counts get more intense colors
        color_intensity = min(0.9, max(0.2, row['count'] / max_count))
        
        # Create a circle marker
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=bubble_radius,
            popup=f"<b>Pincode:</b> {row['pincode']}<br>"
                  f"<b>Count:</b> {row['count']}",
            tooltip=f"Pincode: {row['pincode']}",
            fill=True,
            fill_opacity=color_intensity,
            color='blue',
            fill_color='blue',
            weight=2
        ).add_to(marker_cluster)
    
    # Add a legend (as a simple HTML snippet)
    legend_html = '''
    <div style="position: fixed; 
                bottom: 50px; right: 50px; width: 150px; height: 90px; 
                border:2px solid grey; z-index:9999; font-size:14px;
                background-color:white;
                padding: 10px;
                border-radius: 5px;
                ">
      <span style="font-weight: bold;">Pincode Count</span><br>
      <i class="fa fa-circle" style="color:blue; opacity:0.9;"></i> High<br>
      <i class="fa fa-circle" style="color:blue; opacity:0.5;"></i> Medium<br>
      <i class="fa fa-circle" style="color:blue; opacity:0.2;"></i> Low
    </div>
    '''
    m.get_root().html.add_child(folium.Element(legend_html))
    
    # Add layer control
    folium.LayerControl().add_to(m)
    
    return m

# Create the Folium map with default OpenStreetMap tiles
folium_map = create_folium_map(geocoded_df)

# Display the map
folium_map


In [14]:
# Create a Folium map with Cartodb dark tiles
dark_map = create_folium_map(geocoded_df, tiles='cartodbdark_matter')

# Display the dark map
dark_map

In [15]:
folium_map.save('pincode_bubble_map_light.html')
dark_map.save('pincode_bubble_map_dark.html')
print("Maps saved as HTML files:")
print("- pincode_bubble_map_light.html")
print("- pincode_bubble_map_dark.html")

Maps saved as HTML files:
- pincode_bubble_map_light.html
- pincode_bubble_map_dark.html
